In [342]:
%reload_ext autoreload
%autoreload 2


In [343]:
import numpy as np
from scipy import stats
import pandas as pd
import math
import os
from datetime import date

import sys

from Classes import DataO, MapO, GC




In [344]:
#1. set paths for raw data retreaval and result storing

today = date.today()

# dd/mm/YY
d1 = today.strftime("%d%m%Y")

resultpath="../6_Results/"+d1+"_Maps/"

In [345]:
#2. Gen thresholds and correlation tables

#### THRESHOLDS #####
from numpy import NaN


thresh_df=pd.DataFrame(
    columns=["Maxtemp",
            "Avgtemp"],
    index= ["Gas",
            "Nuclear",
            "Hydroelectric",
            "Solar PV",
            "Transformers",
            "Poles",
            "Overhead Lines",
            "Demand"]
)

thresh_df['Maxtemp']=[16,16,16,25,20,16,23.1,NaN]
thresh_df['Avgtemp']=[NaN,NaN,NaN,NaN,NaN,NaN,NaN,15]

'TESTING'
print(thresh_df)

#### CORRELATIONS #####
correl_df = pd.DataFrame(
    columns=["Maxtemp",
            "Avgtemp"],
    index= ["Gas",
            "Nuclear",
            "Hydroelectric",
            "Solar PV",
            "Transformers",
            "Poles",
            "Overhead Lines",
            "Demand"]
)

correl_df.loc["Gas","Maxtemp"]= lambda x : 0.7587*x-12.1
correl_df.loc["Nuclear","Maxtemp"]= lambda x : 0.7587*x-12.1
correl_df.loc["Hydroelectric","Maxtemp"]= lambda x : 0.7587*x-12.1
correl_df.loc["Solar PV","Maxtemp"]= lambda x : 0.48*(-x+25)
correl_df.loc["Transformers","Maxtemp"]= lambda x : 1*(-x+20)
correl_df.loc["Poles","Maxtemp"]= lambda x : 0.7587*x-12.1
correl_df.loc["Overhead Lines","Maxtemp"]= lambda x : 1.3*x
correl_df.loc["Demand","Avgtemp"]= lambda x : 3.7*(-x+15)

print(correl_df)


                Maxtemp  Avgtemp
Gas                16.0      NaN
Nuclear            16.0      NaN
Hydroelectric      16.0      NaN
Solar PV           25.0      NaN
Transformers       20.0      NaN
Poles              16.0      NaN
Overhead Lines     23.1      NaN
Demand              NaN     15.0
                                              Maxtemp  \
Gas             <function <lambda> at 0x7fdc85156940>   
Nuclear         <function <lambda> at 0x7fdca2656940>   
Hydroelectric   <function <lambda> at 0x7fdca2656ee0>   
Solar PV        <function <lambda> at 0x7fdc8513f310>   
Transformers    <function <lambda> at 0x7fdc8513f280>   
Poles           <function <lambda> at 0x7fdc8513fc10>   
Overhead Lines  <function <lambda> at 0x7fdc8513fa60>   
Demand                                            NaN   

                                              Avgtemp  
Gas                                               NaN  
Nuclear                                           NaN  
Hydroelectric        

In [346]:
#3. Initialises Data objects

#setting resolutions

spatial_res=60
time_res='daily'
start_year=2021
finish_year=2041 #finish year is included

#creating result directory
if not os.path.exists(resultpath):
    os.mkdir(resultpath)

#all data objects are created with all available data members in directory (fdpath)
#initialising runs data loading and coordinate vectors loading

Maxtemp=DataO.DataObject(title="60_Daily_Maxtemp_2021_2041_UK",
                        varID="Maxtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7200,
                        fdpath="../3_Raw_Data/Maxtemp/All",
                        respath=resultpath)

Avgtemp=DataO.DataObject(title="60_Daily_Avgtemp_2021_2041_UK",
                        varID="Avgtemp",sres=spatial_res,tres=time_res,syear=start_year,fyear=finish_year,
                        members=12,
                        total_rows=7200,
                        fdpath="../3_Raw_Data/Avgtemp/All",
                        respath=resultpath)

datao_list=[Maxtemp, Avgtemp]
'TESTING'

for object in datao_list:
    assert object.untreated_array.shape == (12,7200,1,75), 'Loaded data array does not have expected shape'

In [347]:
#4. Statistics test and calculations

Maxtemp.run_stats()
Avgtemp.run_stats()

max_temps=[Maxtemp.member_max, Avgtemp.member_max] #collecting each location's max temperature across time


In [348]:
#5. Initlises Map objects and builds max 20y value maps for Maxtemp and Avgtemp
Map_60_Daily_2021_2040=MapO.MapObject_daily(MapID="Maxtemp and Avtemp - max value in 20y",respath=resultpath,coord_dict=Maxtemp.coord_dict,geojsontitle='max_temps_timewise')


time_max=np.amax(max_temps[0],axis=0)
ftime_max=time_max.flatten('F')
Map_60_Daily_2021_2040.geojson_props('maxtemp - 20y max',ftime_max)

time_max=np.amax(max_temps[1],axis=0)
ftime_max=time_max.flatten('F')
Map_60_Daily_2021_2040.geojson_props('avgtemp - 20y max',ftime_max)

Map_60_Daily_2021_2040.geojson_write('max_temps_timewise') #contains timewise max for avg and maxtemps in 202021_2040

In [349]:
#6. Initialises Grid Component Objects

Gas=GC.GComponent("Gas")
Nuclear=GC.GComponent("Nuclear")
Hydro=GC.GComponent("Hydroelectric")
SolarPV=GC.GComponent("Solar PV")
Transf=GC.GComponent("Transformers")
Poles=GC.GComponent("Poles")
OLines=GC.GComponent("Overhead Lines")
Demand=GC.GComponent("Demand")

'WARNING: DEMAND has correl in avgtemp not in maxtemp, will trigger bug'
component_list=[Gas,Nuclear,Hydro,SolarPV,Transf,Poles,OLines,Demand]

datao_list=[Maxtemp, Avgtemp]

for gc in component_list:
    for wv in datao_list:

        print('\n component ID: ' + gc.GCID)

        gc.init_thresh(thresh_df)
        gc.init_correl(correl_df,datao_list)
        
        #6. Prompts impact array generation
        thresh=gc.thresholds.loc[wv.varID]
        gc.calc_impacts(thresh,wv)

    fmax_impact_array=gc.impact_arrays["max impact"].flatten('F')

    'TODO: initialise correl for demand or enable passing when no correl in table'
        
    Map_60_Daily_2021_2040.geojson_props('impact: -'+ gc.GCID + ' ' ,fmax_impact_array)

Map_60_Daily_2021_2040.geojson_write('impact Maxtemp')


 component ID: Gas

in calc_impacts, printing for: Maxtemp

 component ID: Gas

in calc_impacts, printing for: Avgtemp
no correlations provided, impact calculation unsuccesful

 component ID: Nuclear

in calc_impacts, printing for: Maxtemp

 component ID: Nuclear

in calc_impacts, printing for: Avgtemp
no correlations provided, impact calculation unsuccesful

 component ID: Hydroelectric

in calc_impacts, printing for: Maxtemp

 component ID: Hydroelectric

in calc_impacts, printing for: Avgtemp
no correlations provided, impact calculation unsuccesful

 component ID: Solar PV

in calc_impacts, printing for: Maxtemp

 component ID: Solar PV

in calc_impacts, printing for: Avgtemp
no correlations provided, impact calculation unsuccesful

 component ID: Transformers

in calc_impacts, printing for: Maxtemp

 component ID: Transformers

in calc_impacts, printing for: Avgtemp
no correlations provided, impact calculation unsuccesful

 component ID: Poles

in calc_impacts, printing for: Maxte